<a href="https://colab.research.google.com/github/felanbi/Projet-OC-Categoriser-automatiquement-des-questions/blob/main/P5_01_notebookexploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
path_queries = 'drive/MyDrive/data/'

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import IPython.display
import os
import nltk
import re
import string
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import FreqDist

In [44]:
import nltk 

nltk.download("wordnet")
nltk.download('averaged_perceptron_tagger')
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
sql_query = '''
SELECT
  Id,
  Title,
  Body,
  Tags
FROM 
  Posts
WHERE 
  Tags != ''
  and CreationDate >= '2020-01-01' 
  and Score > 0
  and ViewCount >= 100
  and AnswerCount > 0
'''

In [38]:
data = pd.DataFrame()

print('Lecture des fichiers:\n')
for f in os.listdir(path_queries):
    if 'query' in f:
        print('Fichier en cours de chargement: ' + f)
        data = pd.concat([data,
                          pd.read_csv(path_queries + f, index_col = 0)])

data.columns = [col.lower() for col in data.columns]
data.head()

Lecture des fichiers:

Fichier en cours de chargement: query_2021_1.csv
Fichier en cours de chargement: query_2021_2.csv
Fichier en cours de chargement: query_2020_1.csv
Fichier en cours de chargement: query_2020_2.csv
Fichier en cours de chargement: query_2020_3.csv
Fichier en cours de chargement: query_2020_4.csv
Fichier en cours de chargement: query_2020_5.csv
Fichier en cours de chargement: query_2020_6.csv
Fichier en cours de chargement: query_2020_7.csv
Fichier en cours de chargement: query_2020_8.csv


,title,body,tags
Id,,,
65526399,Groovy script code to append xml node taking 1...,<p>Below code is working but taking 15+ hours ...,<javascript><xml><groovy><xml-parsing><sap-clo...
65526401,How do I make a class instance using user input?,<p>I am making a text based adventure game in ...,<python>
65526407,"How to remove Branding WHMCS Ver 8.1 ""Powered ...",<p>I am a newbie and this is the first time ev...,<php><jquery><css><templates><whmcs>
65526419,How can I construct my objects allocated throu...,<p>C++20 removed the <code>construct()</code> ...,<c++><std><c++20><allocator>
65526447,React Router v5.2 - Blocking route change with...,<p>My app has two pages: <code>Step1</code> an...,<javascript><reactjs><react-router><react-rout...


In [39]:
df = data.copy()

df.tags = df.tags.map(lambda x: x[1:-1].replace('><', ' ').split())

tags_extended = [x for liste in df.tags for x in liste]
tags = set(tags_extended)

# top 20 tags
top_tags = [x[0] for x in FreqDist(tags_extended).most_common(20)]
df.tags = df.tags.map(lambda x: [t for t in x if t in top_tags])

df = df[df.tags.map(len) != 0]
df = df.head(80000)
df.head()

,title,body,tags
Id,,,
65526399,Groovy script code to append xml node taking 1...,<p>Below code is working but taking 15+ hours ...,[javascript]
65526401,How do I make a class instance using user input?,<p>I am making a text based adventure game in ...,[python]
65526407,"How to remove Branding WHMCS Ver 8.1 ""Powered ...",<p>I am a newbie and this is the first time ev...,"[php, css]"
65526419,How can I construct my objects allocated throu...,<p>C++20 removed the <code>construct()</code> ...,[c++]
65526447,React Router v5.2 - Blocking route change with...,<p>My app has two pages: <code>Step1</code> an...,"[javascript, reactjs]"


In [40]:
def clean_text(text, tags, html = False):
    if html:
      # Getting text from raw html file if it is
      from bs4 import BeautifulSoup
      soup = BeautifulSoup(text, 'html.parser')
      text = soup.get_text()
    
    # Lowering all words in text
    text = text.lower()
    
    # On sépare le texte pour ne garder que les tags et le texte sans les tags
    text_split = text.split()
    tags_in_text = [w for w in text_split if w in tags]
    text = [w for w in text_split if w not in tags]
    
    # Removing non letters (unless if word is not in tags) :
    text = ' '.join(text)
    text = re.sub('[^a-zA-Z]', ' ', text) 
        
    return text + ' '.join(tags_in_text) 

df.title = df.title.map(lambda x: clean_text(x, top_tags))
df.body = df.body.map(lambda x: clean_text(x, top_tags, html = True))

df['post'] = df.title + df.body
df.drop(columns = ['title', 'body'], inplace = True)
df = df[['post', 'tags']]

df.head()

,post,tags
Id,,
65526399,groovy script code to append xml node taking ...,[javascript]
65526401,how do i make a class instance using user inpu...,[python]
65526407,how to remove branding whmcs ver powered ...,"[php, css]"
65526419,how can i construct my objects allocated throu...,[c++]
65526447,react router v blocking route change with...,"[javascript, reactjs]"


In [43]:
def tokenizer(post, tags, lemmatizer = False, without_stopwords = False):

    from nltk.tokenize import word_tokenize

    tags_in_post = [tag for tag in post.split()]
    post = [w for w in post.split() if w not in tags]
    post = ' '.join(post)

    token = word_tokenize(post)

    return " ".join(token + tags_in_post)

df.post = df.post.map(lambda x: tokenizer(x, top_tags))
df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,post,tags
Id,,
65526399,groovy script code to append xml node taking h...,[javascript]
65526401,how do i make a class instance using user inpu...,[python]
65526407,how to remove branding whmcs ver powered by wh...,"[php, css]"
65526419,how can i construct my objects allocated throu...,[c++]
65526447,react router v blocking route change with crea...,"[javascript, reactjs]"


In [48]:
from nltk.stem import SnowballStemmer
from nltk import pos_tag

stemmer = SnowballStemmer("english")

def stem_text(x):
  return [stemmer.stem(w) for w in x.split()]

df.post = df.post.map(stem_text)

def remove_verbs(x):
  x = pos_tag(x)
  
  return [w[0] for w in x if w[1] not in {"VBD", "VB"}]

df.post = df.post.map(remove_verbs)

In [49]:
df.to_csv(path_queries + "data_cleaned_13032022", index = True)

In [56]:
df = pd.read_csv(path_queries + "data_cleaned_13032022")
df.set_index("Id", inplace = True)

def tolist(x):
    x = x[2:-2]
    return x.split("', '")

df.post = df.post.map(tolist)
df.tags = df.tags.map(tolist)

df.head()

,post,tags
Id,,
65526399,"[groovi, script, code, to, xml, node, take, ho...",[javascript]
65526401,"[how, do, i, a, class, instanc, use, user, inp...",[python]
65526407,"[how, to, brand, whmcs, ver, power, by, whmcom...","[php, css]"
65526419,"[how, can, construct, my, object, alloc, throu...",[c++]
65526447,"[react, router, v, block, rout, chang, with, c...","[javascript, reactjs]"


In [50]:
from nltk.corpus import stopwords

stopwords_file = open(path_queries + "stopwords.txt")
stopwords = {x.replace("\n", "") for x in stopwords_file.readlines()}
stopwords.update(nltk_sw)

def remove_sw(x, tags):
  text_tags = [w for w in x if w in tags]
  text = [w for w in x if w not in tags]
  text = [w for w in text if w not in stopwords]
  text.extend(text_tags)

  return text 

df.post = df.post.map(remove_sw)

In [52]:
stopwords

{'a',
 'about',
 'above',
 'across',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyone',
 'anything',
 'anywhere',
 'are',
 'area',
 'areas',
 'aren',
 "aren't",
 'around',
 'as',
 'ask',
 'asked',
 'asking',
 'asks',
 'at',
 'away',
 'b',
 'back',
 'backed',
 'backing',
 'backs',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'been',
 'before',
 'began',
 'behind',
 'being',
 'beings',
 'below',
 'best',
 'better',
 'between',
 'big',
 'both',
 'but',
 'by',
 'c',
 'came',
 'can',
 'cannot',
 'case',
 'cases',
 'certain',
 'certainly',
 'clear',
 'clearly',
 'come',
 'could',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'differ',
 'different',
 'differently',
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'done',
 'down',
 'downed',
 'downing',
 'downs',
 'during',
 'e',
 'each',
 'early',
 'either',

In [ ]:
def tostr(x):
  return " ".join(x)

df.post = df.post.map(tostr)

In [ ]:
df.to_csv(path_queries + "data_cleaned_13032022", index = True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(vocabulary = vocabulary)
X = tfidf_vectorizer.fit_transform(df.post)
tfidf_data = pd.DataFrame(X.toarray(), columns = vocabulary)
tfidf_data.head()

,data,error,this,name,code,value,const,user,type,from,...,accesskeyid,fragmentb,svgs,arcgis,rbind,transmit,evening,mistaken,scons,claide
0,0.021006,0.0,0.000000,0.022027,0.033898,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.000000,0.147559,0.000000,0.0,0.000000,0.081427,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.114692,0.000000,0.062527,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.026892,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.070974,0.000000,0.000000,0.0,0.073367,0.034687,0.0,0.011343,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tfidf_data['id'] = df.index.copy()
tfidf_data.set_index('id', inplace = True)
tfidf_data.head()

,data,error,this,name,code,value,const,user,type,from,...,accesskeyid,fragmentb,svgs,arcgis,rbind,transmit,evening,mistaken,scons,claide
id,,,,,,,,,,,,,,,,,,,,,
65526399,0.021006,0.0,0.000000,0.022027,0.033898,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65526401,0.000000,0.0,0.000000,0.147559,0.000000,0.0,0.000000,0.081427,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65526407,0.000000,0.0,0.114692,0.000000,0.062527,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65526419,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.026892,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65526447,0.000000,0.0,0.070974,0.000000,0.000000,0.0,0.073367,0.034687,0.0,0.011343,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
